# TDS Project: Dataset Selection, EDA, and Basic Model Analysis

**Group Members:**
- Adir Elmakais - 316413640

## Introduction

This project aims to analyze the "Student Performance Factors" dataset to understand the various factors influencing students' exam scores. We will perform exploratory data analysis (EDA), build a baseline regression model using XGBoost, and conduct an error analysis to identify areas for improvement.


## Installation Guide

#### Use Python 3.12.0

To get started with the project, ensure you are using **Python 3.12.0**.

1. **Install Python 3.12.0**:
   - Download the installer for Python 3.12.0 from the [official Python website](https://www.python.org/downloads/release/python-3120/).
   - During the installation, make sure to check the box **"Add Python to PATH"**.

2. **macOS: Install `libomp`**:
   - For macOS users, you need to install `libomp` for compatibility with XGBoost. Run the following command:
     ```bash
     brew install libomp
     ```

3. **Install Required Packages**:
   - Once Python 3.12.0 is installed, you can install the necessary packages listed in the `requirements.txt` file by running the following command in your terminal:
     ```bash
     pip install -r requirements.txt
     ```

## 1. Dataset Selection

### Dataset Overview

We have selected the **"Student Performance Factors"** dataset from [Kaggle](https://www.kaggle.com/datasets/lainguyn123/student-performance-factors). This dataset provides comprehensive insights into various factors affecting student performance in exams, including study habits, attendance, parental involvement, and more.

### Dataset Details

- **Number of Records:** 6,607
- **Number of Features:** 20
- **Attribute Types:**
  - **Numeric:** Hours_Studied, Attendance, Sleep_Hours, Previous_Scores, Tutoring_Sessions, Family_Income, Distance_from_Home, Physical_Activity, etc.
  - **Categorical:** Parental_Involvement, Access_to_Resources, Extracurricular_Activities, Motivation_Level, Internet_Access, Teacher_Quality, School_Type, Peer_Influence, Learning_Disabilities, Parental_Education_Level, Gender, etc.

### Reference

> **Dataset Reference:** "Student Performance Factors," available on [Kaggle](https://www.kaggle.com/datasets/lainguyn123/student-performance-factors).


## 2. Data Analysis

### a. Data Loading and Inspection

First, we load the dataset and perform initial inspections to understand its structure and contents.

In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setting plot styles
sns.set_theme(style="whitegrid")
%matplotlib inline

# Loading the dataset
data = pd.read_csv('data/StudentPerformanceFactors.csv')

# Displaying the first few rows
data.head()


### b. Data Cleaning

Before diving into analysis, it's essential to check for missing values and handle them appropriately.


In [ ]:
# Checking for missing values
data.isnull().sum()


Upon inspecting the dataset, we identified missing values in the following categorical columns:

- **Teacher_Quality:** 78 missing values
- **Parental_Education_Level:** 90 missing values
- **Distance_from_Home:** 67 missing values

To maintain data integrity without introducing potential biases from imputation, we will **remove all rows** containing missing values in these columns.


In [ ]:
# Displaying the number of missing values before cleaning
print("Missing Values Before Cleaning:")
print(data.isnull().sum())

# Dropping rows with missing values in specified columns
columns_with_missing = ['Teacher_Quality', 'Parental_Education_Level', 'Distance_from_Home']
data_cleaned = data.dropna(subset=columns_with_missing)

# Displaying the number of missing values after cleaning
print("\nMissing Values After Cleaning:")
print(data_cleaned.isnull().sum())

# Displaying the change in dataset size
print(f"\nOriginal dataset size: {data.shape[0]} rows")
print(f"Cleaned dataset size: {data_cleaned.shape[0]} rows")
print(f"Rows removed: {data.shape[0] - data_cleaned.shape[0]}")


### Exploratory Data Analysis (EDA)
### a. Data Loading and Inspection
We begin by loading the cleaned dataset and performing initial inspections to understand its structure and contents.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Setting plot styles
sns.set_theme(style="whitegrid")
%matplotlib inline

# Loading the cleaned dataset
data_cleaned = data_cleaned.copy()  # Ensuring we're working with the cleaned data

# Displaying the first few rows
data_cleaned.head()


### b. Data Summary

Let's explore the basic statistics and data types to gain an overview of the dataset.


In [ ]:
# Displaying dataset information
data_cleaned.info()
data_cleaned.describe()


**Findings:**

- **Numerical Features:** Several attributes like `Hours_Studied`, `Attendance`, `Sleep_Hours`, `Previous_Scores`, etc., are numerical and suitable for quantitative analysis.
- **Categorical Features:** Attributes such as `Parental_Involvement`, `Access_to_Resources`, `Extracurricular_Activities`, etc., are categorical and may require encoding for modeling.
- **No Missing Values:** All missing values have been addressed, ensuring the dataset is complete.


### c. Visualizations

We'll create the following five visualizations:

1. Distribution of Exam_Score
2. Correlation Heatmap
3. Box Plot: Parental_Involvement vs. Exam_Score
4. Scatter Plot: Hours_Studied vs. Exam_Score
5. Pair Plot of Selected Features and Exam_Score

#### 1. Distribution of Exam_Score

Understanding the distribution of the target variable `Exam_Score` is essential to identify its central tendency, spread, and potential skewness.


In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(data_cleaned['Exam_Score'], bins=20, kde=True, color='skyblue')
plt.title('Distribution of Exam Scores')
plt.xlabel('Exam Score')
plt.ylabel('Frequency')
plt.show()


**Explanation:**

The histogram illustrates the frequency distribution of `Exam_Score`. The KDE (Kernel Density Estimate) overlay helps in visualizing the underlying distribution shape, indicating whether the scores are normally distributed, skewed, or exhibit any multi-modality.


#### 2. Correlation Heatmap

A correlation heatmap helps identify the strength and direction of relationships between `Exam_Score` and other numerical features.


In [ ]:
# Selecting numeric features for correlation
numeric_features = data_cleaned.select_dtypes(include=[np.number]).columns.tolist()

# Calculating correlation matrix
corr_matrix = data_cleaned[numeric_features].corr()

plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Numeric Features')
plt.show()


**Explanation:**

The heatmap displays the correlation coefficients between `Exam_Score` and other numerical features. Strong positive or negative correlations can highlight significant predictors for the regression model, aiding in feature selection and engineering.


#### 3. Parental Involvement vs. Exam_Score

This box plot examines how different levels of parental involvement impact students' exam scores.


In [ ]:
plt.figure(figsize=(8,6))
sns.boxplot(x='Parental_Involvement', y='Exam_Score', data=data_cleaned, palette='Set2')
plt.title('Parental Involvement vs. Exam Score')
plt.xlabel('Parental Involvement')
plt.ylabel('Exam Score')
plt.show()


**Explanation:**

The box plot reveals the distribution of `Exam_Score` across different levels of `Parental_Involvement`. It helps in identifying whether higher levels of parental involvement are associated with better exam performance, highlighting potential areas for intervention and support.


#### 4. Hours Studied vs. Exam_Score

A scatter plot to analyze the relationship between the number of hours a student studies and their exam scores.


In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='Hours_Studied', y='Exam_Score', data=data_cleaned, hue='Motivation_Level', palette='viridis')
plt.title('Hours Studied vs. Exam Score')
plt.xlabel('Hours Studied per Week')
plt.ylabel('Exam Score')
plt.legend(title='Motivation Level')
plt.show()


**Explanation:**

The scatter plot visualizes how `Hours_Studied` correlates with `Exam_Score`, with points colored by `Motivation_Level`. This helps in understanding whether more study hours lead to higher scores and how motivation influences this relationship.


#### 5. Pair Plot of Selected Features and Exam_Score

A pair plot to explore interactions between multiple features and their relationship with `Exam_Score`.


In [ ]:
selected_features = ['Hours_Studied', 'Attendance', 'Sleep_Hours', 'Previous_Scores', 'Exam_Score']
sns.pairplot(data_cleaned[selected_features], diag_kind='kde', plot_kws={'alpha':0.5, 's':50})
plt.suptitle('Pair Plot of Selected Features and Exam Score', y=1.02)
plt.show()


**Explanation:**

The pair plot allows us to observe pairwise relationships between selected numerical features and the target `Exam_Score`. Diagonal KDE plots show the distribution of each feature, while scatter plots indicate potential correlations and interactions.


### 3. Basic Model Pipeline

#### a. Defining the Regression Problem

**Objective:** Predict the `Exam_Score` of students based on various factors such as study habits, attendance, parental involvement, and more.

**Target Variable:** `Exam_Score` (Numeric)

**Features:**
- **Numerical:** `Hours_Studied`, `Attendance`, `Sleep_Hours`, `Previous_Scores`, `Tutoring_Sessions`, `Family_Income`, `Distance_from_Home`, `Physical_Activity`, etc.
- **Categorical:** `Parental_Involvement`, `Access_to_Resources`, `Extracurricular_Activities`, `Motivation_Level`, `Internet_Access`, `Teacher_Quality`, `School_Type`, `Peer_Influence`, `Learning_Disabilities`, `Parental_Education_Level`, `Gender`, etc.


#### b. Data Preprocessing

To prepare the data for modeling, we will:

1. **Handle Categorical Variables:** Convert categorical variables into numerical formats using **One-Hot Encoding**.
2. **Feature Scaling:** Although XGBoost is less sensitive to feature scaling, ensuring uniformity can aid in model interpretability.
3. **Train-Test Split:** Divide the data into training and testing sets to evaluate model performance effectively.


In [46]:
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Separating features and target
X = data_cleaned.drop('Exam_Score', axis=1)
y = data_cleaned['Exam_Score']

# Identifying categorical and numeric features
categorical_features = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()

# Converting boolean columns to integers if any
for col in categorical_features:
    if X[col].dtype == 'bool':
        X[col] = X[col].astype(int)

# Defining the preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # Leave numeric features as is
)

# Building the machine learning pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb.XGBRegressor(random_state=42))
])

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


**Explanation:**

- **OneHotEncoder:** Transforms categorical variables into a binary matrix, enabling the model to interpret them numerically.
- **ColumnTransformer:** Applies different preprocessing steps to different columns—in this case, One-Hot Encoding to categorical features and leaving numeric features unchanged.
- **Pipeline:** Streamlines the preprocessing and modeling steps, ensuring that all transformations are applied consistently during training and testing.
- **Train-Test Split:** Allocates **80%** of the data for training and **20%** for testing, providing a balance between model training and evaluation.


#### c. Model Training and Evaluation

We will train the **XGBoost Regressor** using the default configuration to establish a baseline performance. The following metrics will be used to evaluate the model:

- **Mean Absolute Error (MAE):** Measures the average magnitude of errors in a set of predictions, without considering their direction.
- **Mean Squared Error (MSE):** Measures the average of the squares of the errors, giving higher weight to larger errors.
- **Root Mean Squared Error (RMSE):** The square root of MSE, providing error magnitudes in the same units as the target variable.
- **R² Score:** Indicates the proportion of the variance in the dependent variable predictable from the independent variables.


In [ ]:
# Training the model
pipeline.fit(X_train, y_train)

# Making predictions on the test set
y_pred = pipeline.predict(X_test)

# Evaluating the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R² Score: {r2:.2f}")


**Model Performance Metrics:**

- **Mean Absolute Error (MAE):** *1.05*
- **Mean Squared Error (MSE):** *5.95*
- **Root Mean Squared Error (RMSE):** *2.44*
- **R² Score:** *0.62*

**Interpretation:**

- **R² Score:** An R² score of *0.62* indicates that **62%** of the variance in `Exam_Score` is explained by the model. While a higher R² signifies better fit, it's essential to ensure that the model's performance isn't excessively high to allow room for improvement in subsequent project stages.
- **Error Metrics:** The MAE and RMSE values provide insights into the average prediction errors. These metrics help in understanding the model's accuracy and reliability.


### 4. Error Analysis

After establishing a baseline model, it's essential to analyze its errors to identify areas for improvement.


#### a. Residual Analysis

Residuals are the differences between actual and predicted values. Analyzing residuals helps in identifying patterns that the model fails to capture, indicating potential areas where the model can be improved.


In [ ]:
# Calculating residuals
residuals = y_test - y_pred

# Residuals vs. Predicted Exam Scores
plt.figure(figsize=(10,6))
sns.scatterplot(x=y_pred, y=residuals, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs. Predicted Exam Scores')
plt.xlabel('Predicted Exam Score')
plt.ylabel('Residuals')
plt.show()


### Residuals vs. Predicted Exam Scores Plot

The plot above visualizes the residuals against the predicted exam scores. Here's a breakdown of the components:

1. **X-Axis (Predicted Exam Scores):** Represents the predicted values of exam scores from a regression model.
2. **Y-Axis (Residuals):** Represents the residuals, which are the differences between the observed and predicted values.
3. **Residuals Analysis:**
   - Points closer to the red dashed horizontal line (residual = 0) indicate better model predictions.
   - There is a slight funnel shape or heteroscedasticity, with residuals spreading out more for higher predicted scores, suggesting that the model's error may vary across the range of predictions.
4. **Red Dashed Line:** Represents the zero-residual line, where the predicted values perfectly match the observed values.
5. **Point Cloud:** Most data points cluster around the zero-residual line, but there are notable outliers with larger residuals, indicating instances where the model significantly underestimates or overestimates exam scores.

This plot is useful for evaluating the regression model's performance and identifying potential issues like heteroscedasticity or outliers.


#### b. Error Distribution

Analyzing the distribution of residuals helps in understanding whether the errors are normally distributed or exhibit any skewness, which can impact the model's reliability.


In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(residuals, bins=30, kde=True, color='salmon')
plt.title('Distribution of Residuals')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.show()


### Distribution of Residuals

The histogram above visualizes the distribution of residuals from a regression model. Here's what it indicates:

1. **Residuals on X-Axis:** The horizontal axis represents the residuals, which are the differences between observed values and predicted values by the model.
2. **Frequency on Y-Axis:** The vertical axis represents the frequency of residuals, showing how often residuals of a certain value occur.
3. **Shape of Distribution:**
   - The residuals appear to be approximately centered around 0, indicating that the model's predictions are, on average, accurate.
   - The presence of a smooth kernel density estimation (KDE) curve provides an additional visualization of the residual distribution.
   - Most residuals are close to 0, which suggests that the model performs well for most observations.
4. **Spread and Outliers:**
   - The residuals exhibit a slight right-skew (a longer tail to the right), meaning a small number of instances where the model underestimates or overestimates by a larger margin.
   - There are some extreme residuals on both ends (e.g., near -10 and +30), which may represent outliers.

### Interpretation
This plot helps assess the assumptions of regression modeling:
- **Normality:** The residuals are roughly symmetric around 0, which is a good indication of normality but not perfect.
- **Homoscedasticity:** The spread of residuals appears consistent for most values, though some larger residuals exist, suggesting potential issues with heteroscedasticity.

Overall, this plot provides a good diagnostic for evaluating how well the regression model fits the data and identifying any unusual patterns in the residuals.


### c. Feature Importance Using SHAP

While traditional feature importance metrics provided by models like XGBoost offer insights into which features contribute most to the predictions, SHAP (SHapley Additive exPlanations) provides a more detailed and interpretable understanding. SHAP values account for feature interactions and offer both global and local interpretability, making them invaluable for model explanations.


In [ ]:
import shap

# Extracting the preprocessed training data
X_train_preprocessed = pipeline.named_steps['preprocessor'].transform(X_train)

# Retrieving feature names after One-Hot Encoding
onehot_features = pipeline.named_steps['preprocessor'].named_transformers_['onehot'].get_feature_names_out(categorical_features)
all_features = list(onehot_features) + numeric_features

# Converting to DataFrame for SHAP
X_train_preprocessed_df = pd.DataFrame(X_train_preprocessed, columns=all_features)

# Creating the SHAP explainer
explainer = shap.TreeExplainer(pipeline.named_steps['model'])

# Calculating SHAP values for the training set
shap_values = explainer.shap_values(X_train_preprocessed_df)

# Bar plot for global feature importance
plt.figure(figsize=(12,8))
shap.summary_plot(shap_values, X_train_preprocessed_df, plot_type="bar", feature_names=all_features, show=False)
plt.title('SHAP Feature Importance (Bar Plot)')
plt.tight_layout()
plt.show()

# Detailed summary plot
shap.summary_plot(shap_values, X_train_preprocessed_df, feature_names=all_features)


## SHAP Analysis Outcomes

### 1. Bar Plot: SHAP Feature Importance
The first plot illustrates the global feature importance using SHAP values. Key takeaways:
- **Attendance** has the highest contribution to the model predictions, indicating its significant influence on the target outcome.
- **Hours_Studied** and **Previous_Scores** are also critical predictors, showing that academic efforts and past performance strongly affect the target variable.
- **Tutoring_Sessions** and **Access_to_Resources_High** play a moderate role in the model's predictions.
- Features like **Motivation_Level_High**, **Parental_Involvement_High**, and **Peer_Influence_Positive** demonstrate smaller but still notable contributions.
- Features such as **Peer_Influence_Negative**, **Parental_Education_Level_Postgraduate**, and **Motivation_Level_Low** exhibit minimal importance.

### 2. Summary Plot: SHAP Value Distribution
The second plot provides insights into both the magnitude and direction of the feature impacts:
- **Attendance**: High SHAP values for attendance (blue points moving to the right) positively impact the model, while low attendance (red points to the left) negatively influences the prediction.
- **Hours_Studied**: More hours studied (red points) drive the model output positively, while fewer hours (blue points) contribute negatively.
- **Previous_Scores**: High scores in the past (red points) lead to better predictions, and low past scores (blue points) reduce the target output.
- **Tutoring_Sessions**: More tutoring sessions (red) positively affect the model output, while fewer sessions (blue) have a negative impact.
- **Access_to_Resources_High**: Students with high resource access (red) perform better, whereas those with low access (blue) have reduced output.

### Feature Value Colors:
- **Blue** indicates low feature values, and **red** indicates high feature values.
- The distribution highlights that positive SHAP values push the model predictions higher, while negative values lower them.

### Implications:
- Strategies to improve outcomes should focus on increasing attendance, study hours, and access to resources.
- Targeted interventions like tutoring and enhancing parental involvement can significantly boost performance.
- Identifying and addressing negative influences, such as low motivation or poor peer influence, can also improve the target outcomes.


#### d. Performance on Subgroups

Evaluating model performance across different categories helps in identifying potential biases or areas where the model performs inconsistently.


In [ ]:
# Adding predictions to the test set
test_data = X_test.copy()
test_data['Actual_Exam_Score'] = y_test
test_data['Predicted_Exam_Score'] = y_pred
test_data['Residual'] = residuals

# Function to evaluate performance by subgroup
def evaluate_subgroup(subgroup, subgroup_name):
    subgroup_data = test_data[test_data[subgroup] == subgroup_name]
    if len(subgroup_data) == 0:
        print(f"**{subgroup} = {subgroup_name}**")
        print("No data available for this subgroup.\n")
        return
    mae = mean_absolute_error(subgroup_data['Actual_Exam_Score'], subgroup_data['Predicted_Exam_Score'])
    mse = mean_squared_error(subgroup_data['Actual_Exam_Score'], subgroup_data['Predicted_Exam_Score'])
    r2 = r2_score(subgroup_data['Actual_Exam_Score'], subgroup_data['Predicted_Exam_Score'])
    print(f"**{subgroup} = {subgroup_name}**")
    print(f"MAE: {mae:.2f}")
    print(f"MSE: {mse:.2f}")
    print(f"R² Score: {r2:.2f}\n")

# Example: Evaluating by 'Parental_Involvement'
print("Model Performance by Parental Involvement Level:")
for level in data_cleaned['Parental_Involvement'].unique():
    evaluate_subgroup('Parental_Involvement', level)

# Example: Evaluating by 'School_Type'
print("Model Performance by School Type:")
for school_type in data_cleaned['School_Type'].unique():
    evaluate_subgroup('School_Type', school_type)


### Model Performance by Subgroups

The following tables summarize the model's performance metrics for different subgroups based on key categorical features:

---

#### **Model Performance by Parental Involvement Level**
This analysis evaluates the model's performance for different levels of parental involvement:

| Parental Involvement Level | MAE  | MSE  | R² Score |
|----------------------------|------|------|----------|
| **Low**                   | 1.30 | 9.77 | 0.43     |
| **Medium**                | 0.85 | 2.70 | 0.79     |
| **High**                  | 1.19 | 8.54 | 0.51     |

- **MAE (Mean Absolute Error):** Indicates the average magnitude of errors in predictions.
- **MSE (Mean Squared Error):** Penalizes larger errors by squaring the residuals.
- **R² Score:** Reflects how well the model explains variance in the actual scores. Higher values indicate better performance.

---

#### **Model Performance by School Type**
This analysis evaluates the model's performance for different types of schools:

| School Type | MAE  | MSE  | R² Score |
|-------------|------|------|----------|
| **Public**  | 1.04 | 5.26 | 0.63     |
| **Private** | 1.05 | 7.38 | 0.60     |

---

### Insights
- The model performs best for the **Medium** parental involvement level, with the lowest MAE and MSE, and the highest R² Score (0.79).
- The **Public** school type achieves slightly better performance than **Private** schools, with a lower MSE and higher R² Score.
- **Low** and **High** parental involvement levels show weaker performance, suggesting areas for potential improvement in the model's predictive ability for these subgroups.


#### e. Overestimation vs. Underestimation

Assessing whether the model tends to overestimate or underestimate exam scores provides insights into potential biases.


In [ ]:
# Categorizing predictions
test_data['Error_Type'] = test_data['Residual'].apply(lambda x: 'Underestimation' if x < 0 else 'Overestimation')

# Counting error types
error_counts = test_data['Error_Type'].value_counts()

# Plotting
plt.figure(figsize=(8,6))
sns.barplot(x=error_counts.index, y=error_counts.values, palette='coolwarm')
plt.title('Overestimation vs. Underestimation')
plt.xlabel('Error Type')
plt.ylabel('Count')
plt.show()

# Calculating percentages
total_errors = error_counts.sum()
over_percent = (error_counts.get('Overestimation', 0) / total_errors) * 100
under_percent = (error_counts.get('Underestimation', 0) / total_errors) * 100
print(f"Overestimation: {over_percent:.2f}%")
print(f"Underestimation: {under_percent:.2f}%")



The bar plot below shows the distribution of errors categorized as **Overestimation** and **Underestimation**. This analysis provides insights into potential biases in the model's predictions.

---

#### Plot Description
- **X-Axis (Error Type):** Shows the two categories of error:
  - **Underestimation:** The model predicted a value lower than the actual exam score.
  - **Overestimation:** The model predicted a value higher than the actual exam score.
- **Y-Axis (Count):** Represents the number of instances for each error type.
- The color scheme (`coolwarm` palette) distinguishes between the two error categories for better interpretability.

---

#### Observations
- **Underestimation:** Accounts for **53.84%** of the errors, suggesting a slight tendency of the model to predict lower than the actual exam scores.
- **Overestimation:** Represents **46.16%** of the errors, indicating that while the model occasionally overpredicts, it does so less frequently compared to underestimations.

---

#### Insights
- The imbalance between underestimation and overestimation, while not drastic, could suggest a slight bias in the model. This could be due to the distribution of target values or other factors influencing the predictions.
- The overall percentages of error types (53.84% underestimation, 46.16% overestimation) are relatively balanced, indicating that the model does not have a significant skew in its predictions.

---

#### Save the Model and Cleaned Data

To ensure reproducibility and ease of access, the following steps save the trained pipeline and export the cleaned dataset:
- The **cleaned dataset** is saved in the `data` folder as `StudentPerformaceFactorsClean.csv`.
- The trained **pipeline (including preprocessing and XGBoost model)** is saved in the `models` folder as `pipeline_model.joblib`.

These folders are created automatically if they do not already exist.

In [ ]:
import os
import joblib

# Create directories if they don't exist
os.makedirs('data', exist_ok=True)
os.makedirs('models', exist_ok=True)

# Save the cleaned data to the data folder
cleaned_data_path = os.path.join('data', 'StudentPerformaceFactorsClean.csv')
data_cleaned.to_csv(cleaned_data_path, index=False)
print(f"Cleaned dataset saved at '{cleaned_data_path}'.")

# Save the pipeline (including preprocessing and XGBoost model) to the models folder
model_path = os.path.join('models', 'pipeline_model.joblib')
joblib.dump(pipeline, model_path)
print(f"Trained pipeline saved at '{model_path}'.")

**References:**

- Kaggle
- ChatGPT